In [ ]:
import sys
!python3 -V

In [1]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade opencv-python
!{sys.executable} -m pip install pyqtwebengine==5.12.0
!{sys.executable} -m pip install pyqt5==5.12.0
!{sys.executable} -m pip install typed-ast==1.3.0
!{sys.executable} -m pip install --upgrade imageai --ignore-installed certifi
!{sys.executable} -m pip install keras==2.3.1
!{sys.executable} -m pip install tensorflow==2.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 MB 5.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.5.64
    Uninstalling opencv-python-4.5.5.64:
      Successfully uninstalled opencv-python-4.5.5.64
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 7.3 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 7.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 5.5 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, but you have pyqt5 5.15.7 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 6.3 MB/s eta 0:00:0000:0100:01m
ERROR: Could not find 

  Using cached imageai-2.1.6-py3-none-any.whl (160 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 5.4 MB/s eta 0:00:00
  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)
  Using cached opencv_python-4.6.0.66-cp36-abi3-macosx_10_15_x86_64.whl (46.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 8.5 MB/s eta 0:00:0000:0100:01m
  Using cached matplotlib-3.3.2.tar.gz (37.9 MB)
  Preparing metadata (setup.py) ... done
  Using cached Pillow-7.0.0.tar.gz (38.2 MB)
  Preparing metadata (setup.py) ... done
  Using cached keras-resnet-0.2.0.tar.gz (9.3 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 kB 6.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Using cached scipy-1.4.1.tar.gz (24.6 MB)
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [2519 lines of o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.8/377.8 kB 6.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.0 requires keras<2.10.0,>=2.9.0rc0, but you have keras 2.3.1 which is incompatible.
ERROR: Could not find a version that satisfies the requirement tensorflow==2.2.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0)
ERROR: No

**1) Bibliotecas:**

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from imageai.Detection import ObjectDetection

**2) Função para extração de frames de videos:**

In [ ]:
def extract_frames(video_path, fr=2):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    id_frame = 0
    while id_frame<=total_frames:
        cap.set(1, id_frame)
        success, frame = cap.read()
        if type(frame) != type(None):
            frames.append(frame)
        id_frame+=fr
    return frames

**3) Extrair frames:**

In [ ]:
frames = extract_frames('/Users/m04m/Downloads/Frutas.avi')
len(frames)

**4) Apresentar um frame para validação:**

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(cv2.cvtColor(frames[0], cv2.COLOR_BGRA2RGB))
plt.axis('off')

**5) Iniciar o modelo yolo:**

In [ ]:
detector = ObjectDetection()
detector.setModelTypeAsTinyYOLOv3()
detector.setModelPath("yolo-tiny.h5")
detector.loadModel()
custom_objects = detector.CustomObjects(person=True)

**6) Reconhecimento de pessoa por frame:**

In [ ]:
rect_h = 8
final_img = []
for img in tqdm(frames):

    detected_image_array, detections = detector.detectCustomObjectsFromImage(
        input_image=img, input_type="array", output_type="array",
        custom_objects=custom_objects, minimum_percentage_probability=75)
    
    new_img = np.zeros(img.shape)
    for person in detections:
        y_min = person['box_points'][1]
        x_min = person['box_points'][0]
        y_max =  person['box_points'][3]
        x_max =  person['box_points'][2]
        cv2.rectangle(new_img, (x_min,y_max-rect_h), (x_max,y_max), (0,255,0), -1)
        #cv2.line(new_img, (x_min,y_max-rect_h), (x_max,y_max), (0,255,0), 4)
        
    final_img.append(new_img[:,:,1]/255)

**7) Calculo de frequencias**

In [ ]:
combi_img = np.sum(final_img, axis=0)
combi_img[combi_img.shape[0] - int(rect_h+rect_h/3):,:] = 0

**8) Mapa de frequencias (tracking)**

In [ ]:
track_img = (combi_img != 0) +0 

plt.figure(figsize=(12,12))
plt.imshow(track_img)
plt.axis('off')

**9) Mapa heatmap**

In [ ]:
img_background = cv2.cvtColor(frames[0], cv2.COLOR_BGRA2RGB)

plt.figure(figsize=(12,12))
plt.imshow(img_background)
plt.imshow(combi_img, cmap='jet', alpha=0.5)
plt.axis('off')